In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:

# Load the CSV file into a DataFrame
data = pd.read_csv("../data/train.csv",dtype={'StateHoliday': str})

In [3]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')


In [4]:
data.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


In [7]:
print(data.dtypes)

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday      int64
SchoolHoliday     int64
dtype: object


In [6]:
# Define the mapping dictionary
holiday_mapping = {"a": 1, "b": 2, "c": 3, "0": 0}

# Map the values in the "StateHoliday" column to integers
data["StateHoliday"] = data["StateHoliday"].map(holiday_mapping)

In [9]:
# Check for NaN values
print(data.isna().sum())


Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64


In [12]:
data['Date'] = pd.to_datetime(data['Date'])
data['DayOfWeek'] = pd.to_datetime(data['Date']).dt.dayofweek
# Extract weekdays and weekends
data['Weekday'] = data['DayOfWeek'].apply(lambda x: 1 if x < 5 else 0)
data['Weekend'] = data['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
# Extract beginning, mid, and end of the month
data['BeginningOfMonth'] = data['Date'].dt.day.apply(lambda x: 1 if x <= 10 else 0)
data['MidMonth'] = data['Date'].dt.day.apply(lambda x: 1 if x > 10 and x <= 20 else 0)
data['EndOfMonth'] = data['Date'].dt.day.apply(lambda x: 1 if x > 20 else 0)

In [13]:
data.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Weekday,Weekend,BeginningOfMonth,MidMonth,EndOfMonth
0,1,4,2015-07-31,5263,555,1,1,0,1,1,0,0,0,1
1,2,4,2015-07-31,6064,625,1,1,0,1,1,0,0,0,1
2,3,4,2015-07-31,8314,821,1,1,0,1,1,0,0,0,1


In [14]:
columns_to_exclude = ['Date']
columns_to_scale = [column for column in data.columns if column not in columns_to_exclude]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[columns_to_scale])
data[columns_to_scale] = scaled_data

In [15]:
data.head(3)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Weekday,Weekend,BeginningOfMonth,MidMonth,EndOfMonth
0,-1.731640,0.501484,2015-07-31,-0.132683,-0.168269,0.452399,1.273237,-0.159217,2.144211,0.630672,-0.630672,-0.700515,-0.700517,1.388213
1,-1.728534,0.501484,2015-07-31,0.075373,-0.017540,0.452399,1.273237,-0.159217,2.144211,0.630672,-0.630672,-0.700515,-0.700517,1.388213
2,-1.725427,0.501484,2015-07-31,0.659800,0.404499,0.452399,1.273237,-0.159217,2.144211,0.630672,-0.630672,-0.700515,-0.700517,1.388213
